<a href="https://colab.research.google.com/github/ImitatedSky/Python_AI/blob/main/news_sentiment_analysis_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [150]:
from google.colab import drive
import os

# data download from https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news/data

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/dataset') #切換該目錄

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [152]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.layers import SpatialDropout1D, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

In [153]:
header = ["Sentiment", "Contents"]
df = pd.read_csv('sentiment_data.csv', encoding="ISO-8859-1", names=header )


In [154]:
df


,Sentiment,Contents
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [155]:
random_index = np.random.permutation(df.index)
df = df.loc[random_index]

sentiment_mapping = {
    "positive": 2,
    "neutral": 1,
    "negative": 0
}
df['Sentiment'] = df['Sentiment'].map(sentiment_mapping)

In [156]:
df

,Sentiment,Contents
2071,2,Finnish Vacon has signed significant deals wit...
1215,1,"Entire paper mills may be set up , especially ..."
2294,2,"Outokumpu of Finland , stainless steel manufac..."
3616,1,"In 2007 , the Group 's net sales stood at EUR ..."
2215,1,"The company 's profit totaled Ls 134,700 ."
...,...,...
3324,1,The platform would continue to be the developm...
2475,1,Helsinki-based Componenta bought Turkish liste...
1624,1,"The contractor of the shopping center , China ..."
1558,1,LONDON ICIS -- Tikkurila has completed the sal...


In [157]:
# Create  dataframe
# data = df.filter(['Contents'])
# Convert the dataframe to a numpy array
dataset = df.values
# 95%
training_data_len = int(np.ceil( len(dataset) * .95 ))

print(f"總共資料筆數{len(dataset)} , 訓練資料筆數{training_data_len}")

總共資料筆數4846 , 訓練資料筆數4604


In [158]:
train_data = dataset[0:int(training_data_len), :]

x_train = train_data[:,1]
y_train = train_data[:,0]

test_data = dataset[training_data_len : , :]

x_test = test_data[:,1]
y_test = test_data[:,0]


In [159]:
pad_type = trunc_type = 'pre'
max_review_length = 36

In [160]:
# 文本向量化
from keras.preprocessing.text import Tokenizer

# 初始化一個Tokenizer
tokenizer = Tokenizer()

# 適應訓練及數列
tokenizer.fit_on_texts(x_train)
vocab_size = len(tokenizer.word_index)+1
print(vocab_size)

# 將訓練集 測試集 轉為整數序列
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

9877


In [161]:
import tensorflow as tf
from keras.utils import to_categorical

# 再使用pad_sequences函数填充序列
x_train = pad_sequences(x_train, maxlen = max_review_length , padding=pad_type, truncating=trunc_type, value=0, dtype='int32')
x_test = pad_sequences(x_test, maxlen = max_review_length , padding=pad_type, truncating=trunc_type, value=0, dtype='int32')

x_train = tf.convert_to_tensor(x_train, dtype=tf.int32)
y_train = to_categorical(y_train)

x_test = tf.convert_to_tensor(x_test, dtype=tf.int32)
y_test = to_categorical(y_test)

In [162]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPool1D , Dropout, Embedding

model = Sequential()

# 添加嵌入層，將整数序列轉為嵌入向量
model.add(Embedding( input_dim = vocab_size , output_dim = 50 , input_length = max_review_length ))

model.add(SpatialDropout1D( 0.2 ))

model.add(Conv1D(144, 3, activation='relu'))
model.add(Conv1D(144, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(144, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.summary()

# 编译模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=8, batch_size=20)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 36, 50)            493850    
                                                                 
 spatial_dropout1d_6 (Spati  (None, 36, 50)            0         
 alDropout1D)                                                    
                                                                 
 conv1d_18 (Conv1D)          (None, 34, 144)           21744     
                                                                 
 conv1d_19 (Conv1D)          (None, 32, 144)           62352     
                                                                 
 global_max_pooling1d_6 (Gl  (None, 144)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_15 (Dense)            (None, 144)             

In [163]:
predictions = model.predict(x_test)
loss, acc = model.evaluate(x_test, y_test)
print(f"accuracy：{acc}")
print(f"loss：{loss}")

8/8 [==============================] - 0s 9ms/step - loss: 1.3050 - accuracy: 0.7645
accuracy：0.7644628286361694
loss：1.3049994707107544


In [164]:
_input = input("sentiment analysis contents：")
_input = tokenizer.texts_to_sequences([_input])
_input = pad_sequences(_input, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0, dtype='int32')
_input = tf.convert_to_tensor(_input, dtype=tf.int32)

#
predictions = model.predict(_input)

#
sentiment_label = np.argmax(predictions)  # 返回最大的

#
sentiment_mapping = {
    2: "positive",
    1: "neutral",
    0: "negative"
}

predicted_sentiment = sentiment_mapping[sentiment_label]
print("prediction ：", predicted_sentiment)
predictions


sentiment analysis contents：Global Economy Shows Signs of Recovery Amidst Ongoing Challenges
1/1 [==============================] - 0s 41ms/step
prediction ： positive


array([[5.2122277e-04, 4.6474779e-01, 5.3473097e-01]], dtype=float32)